This is my first process made in Python after a course held by [Ujaval Gandhi](https://spatialthoughts.com/). None of this would have been possible without his guidiance.
The input is a list of internships for Urban Landscape engineers in Denmark.

The process checks the input address for typos and geocodes each address. The output is af GeoPackage with point, ready to be used in Qgis.

Both the control for typos and the geocoding is made with the Danish service [DAWA.AWS.DK](https://dawa.aws.dk/) whitch like most geoservices is made 
publicly available.

In [ ]:
import os
import pandas as pd
import geopandas as gpd
import xlrd
import requests
import json
from shapely.geometry import Point
import matplotlib.pyplot as plt

In [ ]:
#data_pkg_path is the subfolder that contains the inputfile
data_pkg_path = 'data'
#filename can be changes to the current input
filename = 'hopi_praktikadresser.xlsx'
path = os.path.join(data_pkg_path, filename)

In [ ]:
#import the excelsheet, sheet_name=1 indicates that is is the second sheet that contains the data. 
df = pd.read_excel(path, sheet_name=1, encoding='utf-8')

In [ ]:
#In the full dataset I have some adresses from other countries so this is to sort only the danish adresses
dk_df = df[df['Postnr.'] < 9000].copy()

In [ ]:
#we make a joined field with a lookup adress
dk_df['xx'] = dk_df['Adresse'] +', ' + dk_df['Postnr.'].map(str)

In [ ]:
#This function control the lookup address and returns a uniqe address, getting the location with the next function and adds columns to the DataFrame.
def find_point(row):
    address = row['xx']
    r = requests.get('https://dawa.aws.dk/datavask/adgangsadresser?betegnelse='+ address)

    if r.status_code == 200:
        data = r.json()
        nummer = data['resultater'][0]['aktueladresse']['vejnavn'], str(data['resultater'][0]['aktueladresse']['husnr'])
        row['uid'] = data['resultater'][0]['aktueladresse']['id']
        row['vejnavn'] = ' '.join( nummer)
        row['postnr'] = data['resultater'][0]['aktueladresse']['postnr']
        row['by'] = data['resultater'][0]['aktueladresse']['postnrnavn']
        dawa_addr = address_from_id(data['resultater'][0]['aktueladresse']['id'])
        row['long'] = dawa_addr["adgangspunkt"]["koordinater"][0]
        row['lat'] = dawa_addr["adgangspunkt"]["koordinater"][1]
        
        return row
    else:
        print('Request failed.')
        return -9999

In [ ]:
#This is taking the uniqe address and finds a location
def address_from_id(uid):
        re = requests.get('https://dawa.aws.dk/adgangsadresser/'+uid)
        return re.json()

In [ ]:
#This is the actual process where every line in the input excelsheet gets a washed address and location. (It takes some time to proces)
dk_df = dk_df.apply(find_point, axis=1)

In [ ]:
#Here we select which columns we need and in the desired order.
renset = dk_df.iloc[:, [0,1,8,9,10,5,11,12]]
geometry=gpd.points_from_xy(renset.long, renset.lat)

gdf = gpd.GeoDataFrame(renset, crs='EPSG:4326', geometry=geometry)

#Just a little look at the result.
gdf.head()

In [ ]:
#finaly the output is written to af GeoPackage for further use in QGIS
output_dir = 'output'
output_filename = 'praktik.gpkg'
output_path = os.path.join(output_dir, output_filename)

gdf.to_file(filename=output_path, layer='praktiklokalitet', driver='GPKG', encoding='UTF-8')
print('Successfully written output file at {}'.format(output_path))


The last code is just a little view of the locations

In [ ]:
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
ax = world[world.name == 'Denmark'].plot(
    color='white', edgecolor='black')

gdf.plot(ax=ax, color='red')

plt.show()